In [1]:
require(data.table)

Loading required package: data.table


### (load usages)

In [2]:
usage = fread("utilities_data/datastore_export/usage.csv", stringsAsFactors=TRUE,
              colClasses=c(parent="factor", key="factor"))
setnames(usage, c("parent", "key"), c("invoice_key", "usage_key"))
setkey(usage, invoice_key, usage_key)

## raw usage count by service and unit

In [3]:
table(usage[, .(unit, service)], useNA="ifany")

                  service
unit                    Electricity Natural Gas  Oil Oil Additional Services
                      0           2           4    0                       0
  100cu.ft.           0           0           0    0                       0
  Gallon              0           0           0    0                       0
  Ton                 0           0           0    0                       0
  W                   0           1           0    0                       0
  bulk                0           0           0    0                       0
  c.f                 0           0           0    0                       0
  cf                  0           0           0    0                       0
  dumpster            0           0           0    0                       0
  each                0           0           0    0                       0
  fl                  0           0           0    0                       0
  gal                 0           0           0   

### (recode Service and Units)
NOTE: "Waste Water" is coded NA (to be removed later) to avoid double-counting

In [4]:
levels(usage$service) = c(
  NA, "Electricity", "Natural Gas", "Oil", NA, "Propane", "Water", "Electricity",
  "Trash", NA, "Water", "Electricity")
levels(usage$unit) = c(
  NA, "cu ft", "gal", "T", "W", NA, "cu ft", "cu ft", NA, NA, NA, "gal",
  "gal", "gal", "gal", "kW", "kWh", "kW", "kWh", "lbs", NA, NA, NA, NA, "thm",
  "thm", "T", "W", NA, "yd", "yd")
table(usage[, .(unit, service)], useNA="ifany")

       service
unit    Electricity Natural Gas  Oil Propane Water Trash <NA>
  cu ft           0           0    0       0   126     0   98
  gal             0          12  104       2   225    57    6
  T               0           0    0       0     0     5    0
  W              36           0    0       0     0     0    0
  kW            606           0    0       0     0     0    0
  kWh          1880           0    0       0     0     1    0
  lbs             0           0    0       0     0    27    0
  thm             0         789    0       0     0     0    0
  yd              0           0    0       0     0   673    2
  <NA>            3           5    0       0    45    86    1

### (load fees)

In [5]:
fee = fread("utilities_data/datastore_export/fee.csv", stringsAsFactors=TRUE,
            colClasses=c(parent="factor", key="factor"))
setnames(fee, c("parent", "key"), c("invoice_key", "fee_key"))
setkey(fee, invoice_key, fee_key)

### (load feeRusage --- the cross table between fees and usages)

In [6]:
feeRusage = fread("utilities_data/datastore_export/feeRusage.csv", stringsAsFactors=TRUE,
                  colClasses="factor")
setkey(feeRusage, invoice_key, fee_key)

## overall fees by service by year

In [7]:
usage[
  setkey(feeRusage[
    fee
  ][
  , .(usage_key, amount=amount / .N), by=key(fee)
  ],
  invoice_key, usage_key)
][
, .(amount=sum(amount)), by=.(start_date=as.integer(strftime(start, "%Y")), service)
][
  order(service, start_date)
]

,start_date,service,amount
1,2008,Electricity,230.92
2,2009,Electricity,1172.49
3,2010,Electricity,715.63
4,2011,Electricity,954
5,2012,Electricity,196531.6
6,2013,Electricity,322578.1
7,2014,Electricity,330148.7
8,2015,Electricity,96719.18
9,2008,Natural Gas,1008.92
10,2009,Natural Gas,3772.82


### (omit Trash and NA usages, drop usages with rate units, then drop unused unit levels)

In [8]:
usage = na.omit(usage)
usage = usage[service != "Trash" & unit != "kW" & unit !="W"]
usage = droplevels(usage)

### (convert units and drop unused unit levels)

In [9]:
cuFtToGal = function(x) 7.48052 * x
usage[unit == "cu ft", `:=`(quantity=cuFtToGal(quantity), unit="gal")]
galGasToThm = function(x) 1.49828783357 * x # assuming residential fuel oil
usage[service == "Natural Gas" & unit == "gal",
      `:=`(quantity=galGasToThm(quantity), unit="thm")]
usage = droplevels(usage)


ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 of list input is not a data.frame, data.table or list


                      end invoice_key     service    meter               start
   1: 2013-11-19T00:00:00      100001 Electricity  3565913 2013-10-21T00:00:00
   2: 2013-11-19T00:00:00      100001 Natural Gas  2029093 2013-10-21T00:00:00
   3: 2013-11-12T00:00:00      100002 Electricity 14042180 2013-10-14T00:00:00
   4: 2013-10-01T00:00:00      100003 Electricity     4557 2013-09-01T00:00:00
   5: 2014-04-14T00:00:00      100004       Water          2014-03-17T00:00:00
  ---                                                                         
3134: 2013-03-26T00:00:00     9300003 Electricity    30891 2013-03-01T00:00:00
3135: 2015-02-28T00:00:00     9300005       Water          2015-02-01T00:00:00
3136: 2012-11-30T00:00:00     9300006       Water          2012-11-01T00:00:00
3137: 2015-01-12T00:00:00     9310002       Water          2014-12-10T00:00:00
3138: 2013-03-31T00:00:00     9310007       Water          2013-03-01T00:00:00
      usage_key unit    quantity
   1:         1  kW

ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 of list input is not a data.frame, data.table or list


                      end invoice_key     service    meter               start
   1: 2013-11-19T00:00:00      100001 Electricity  3565913 2013-10-21T00:00:00
   2: 2013-11-19T00:00:00      100001 Natural Gas  2029093 2013-10-21T00:00:00
   3: 2013-11-12T00:00:00      100002 Electricity 14042180 2013-10-14T00:00:00
   4: 2013-10-01T00:00:00      100003 Electricity     4557 2013-09-01T00:00:00
   5: 2014-04-14T00:00:00      100004       Water          2014-03-17T00:00:00
  ---                                                                         
3134: 2013-03-26T00:00:00     9300003 Electricity    30891 2013-03-01T00:00:00
3135: 2015-02-28T00:00:00     9300005       Water          2015-02-01T00:00:00
3136: 2012-11-30T00:00:00     9300006       Water          2012-11-01T00:00:00
3137: 2015-01-12T00:00:00     9310002       Water          2014-12-10T00:00:00
3138: 2013-03-31T00:00:00     9310007       Water          2013-03-01T00:00:00
      usage_key unit    quantity
   1:         1  kW

## overall usage by service/unit by year

In [10]:
usage[
, .(quantity=sum(quantity)),
  by=.(start_date=as.integer(strftime(start, "%Y")),
       service,
       unit)
][
  order(service, start_date)
]

,start_date,service,unit,quantity
1,2008,Electricity,kWh,2399
2,2009,Electricity,kWh,11734
3,2010,Electricity,kWh,6429
4,2011,Electricity,kWh,8661
5,2012,Electricity,kWh,1839434
6,2013,Electricity,kWh,4072070
7,2014,Electricity,kWh,3030298
8,2015,Electricity,kWh,897594
9,2008,Natural Gas,thm,978
10,2009,Natural Gas,thm,4374
